# Global Tempeature Data

The earth is warming by a lot, but physically it may feel different for someone living in Stockholm than it is for another in Tokyo. This notebook aims to visually understand this better and further inform our project.

In [4]:
# import the dask libraries
import dask
import dask.dataframe as dd
from dask.distributed import LocalCluster, Client

#import graphing libraries
import seaborn as sns

In [2]:
major_cities = dd.read_csv('GlobalTemp/GlobalLandTemperaturesByMajorCity.csv')

In [3]:
major_cities.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1849-01-01,26.704,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W
1,1849-02-01,27.434,1.362,Abidjan,Côte D'Ivoire,5.63N,3.23W
2,1849-03-01,28.101,1.612,Abidjan,Côte D'Ivoire,5.63N,3.23W
3,1849-04-01,26.140,1.387,Abidjan,Côte D'Ivoire,5.63N,3.23W
4,1849-05-01,25.427,1.200,Abidjan,Côte D'Ivoire,5.63N,3.23W


In [ ]:
# Create a Dask Client
cluster = LocalCluster()
client = Client(cluster)
client()

In [ ]:
# close the client
client.close()